In [6]:
pip install torch

     -------------------------------------- 172.4/172.4 MB 5.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install cmfrec

     -------------------------------------- 268.3/268.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached findblas-0.1.21-py3-none-any.whl
  Created wheel for cmfrec: filename=cmfrec-3.5.1.post4-cp39-cp39-win_amd64.whl size=973386 sha256=185471e80eed89a098a9237bc7a7e679d90c9470cc43b3d7e59d7d1d18522861
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\fb\34\7e\98161f512b5008289e510c24b61e69b010bdebc26478273200
Successfully built cmfrec
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing_extensions import dataclass_transform
import pandas as pd
import numpy as np
import scipy.stats
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings(action='ignore')
import pickle
import torch
from sklearn.preprocessing import MinMaxScaler
from cmfrec import CMF, CMF_implicit
import bottleneck as bn
from preprocess import load_data, preprocess_rival

#참고: https://cotak.tistory.com/25
from sqlalchemy import create_engine
import os
import json
import pymysql

In [3]:
def numerize_for_infer(tp, profile2id, show2id):
    uid = tp['handle'].apply(lambda x: profile2id[x])
    sid = tp['problems'].apply(lambda x: show2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [10]:
def make_item_attributes(problem, user_problems, show2id):
    # 문제별 tag 정리
    tags = ['math', 'implementation', 'greedy', 'string', 'data-structures', 'geometry', 'dp', 'graphs']
    problems_tag = problem[['problemId','tags']]
    problems_tag.problem_id = problems_tag.problemId.astype('str')
    problems_tag.tags = problems_tag.tags.apply(lambda x: str(x).split(','))
    problems_tag = problems_tag.explode('tags').reset_index(drop=True)
    problems_tag = problems_tag[problems_tag.tags.isin(tags)]
    problems_tag['values']=1
    problems_tag = problems_tag.pivot_table(index="problemId", columns=["tags"], aggfunc=np.sum, values='values', fill_value=0)

    # 유저가 푼 문제 태그 고르기
    problems_tag.reset_index(inplace=True, drop=False)
    solved_unique = user_problems.problems.unique()
    solved_problems_tag = problems_tag[problems_tag.problemId.isin(solved_unique)]

    # 문제 ID 변환
    solved_problems_tag.problemId = solved_problems_tag.problemId.apply(lambda x: show2id[x])

    # 문제 ID 변환
    solved_problems.problemId = solved_problems.problemId.apply(lambda x: show2id[x])

    # 푼 문제 고르기
    problem.problemId = problem.problemId.astype('str')
    solved_unique = user_problems.problems.unique()
    solved_problems = problem[problem.problemId.isin(solved_unique)]

    # 두 데이터프레임 합치기
    item_attributes = pd.merge(solved_problems, solved_problems_tag, left_on='problemId', right_on='problemId')

    # 불필요한 행 제거
    item_attributes.drop('id', axis=1, inplace=True)
    item_attributes.drop('tags', axis=1, inplace=True)
    item_attributes.drop('title', axis=1, inplace=True)
    item_attributes.drop('isSolvable', axis=1, inplace=True)

    # Minmax scaler를 사용하여 scaling 하기
    accepted_user_scaler = MinMaxScaler()
    avg_tries_scaler = MinMaxScaler()

    item_attributes.acceptedUserCount = accepted_user_scaler.fit_transform(item_attributes.acceptedUserCount.values.reshape(-1, 1))
    item_attributes.averageTries = avg_tries_scaler.fit_transform(item_attributes.averageTries.values.reshape(-1, 1))

    return item_attributes

In [12]:
def make_user_attributes(user_problems, profile2id, users):
    # 푼 유저 고르기
    unique_users = user_problems.handle.unique()
    users = users[users.handle.isin(unique_users)]

    # 유저 ID 변환
    users.handle = users.handle.apply(lambda x: profile2id[x])

    # 불필요한 행 지우기
    users.drop('id', axis=1, inplace=True)
    users.drop('rivalCount', axis=1, inplace=True)
    users.drop('reverseRivalCount', axis=1, inplace=True)
    users.drop('rating', axis=1, inplace=True)
    users.drop('rank', axis=1, inplace=True)
    users.drop('maxStreak', axis=1, inplace=True)

    # Minmax scaler를 사용하여 scaling 하기
    solved_count_scaler = MinMaxScaler()
    rating_by_problems_sum_scaler = MinMaxScaler()
    rating_by_class_scaler = MinMaxScaler()
    rating_by_solved_count_scaler = MinMaxScaler()

    users.solvedCount = solved_count_scaler.fit_transform(users.solvedCount.values.reshape(-1, 1))
    users.ratingByProblemsSum = rating_by_problems_sum_scaler.fit_transform(users.ratingByProblemsSum.values.reshape(-1, 1))
    users.ratingByClass = rating_by_class_scaler.fit_transform(users.ratingByClass.values.reshape(-1, 1))
    users.ratingBySolvedCount = rating_by_solved_count_scaler.fit_transform(users.ratingBySolvedCount.values.reshape(-1, 1))

    return users

In [13]:
def make_lst_rivals(i, df_result):
    return df_result['r1'][i]+','+df_result['r2'][i]+','+df_result['r3'][i]+','+df_result['r4'][i]+','+df_result['r5'][i]+','+df_result['r6'][i]

In [14]:
def first_division(num):
    for i in range(2, num+1):
        if num % i == 0:
            return i
    return 1

In [9]:
def rival_cmf(db):
    problem, solved_problem, users, problems_class = load_data(db)

    user_problems = solved_problem[['handle', 'problems']]
    user_problems.problems = user_problems.problems.str.split(',')
    user_problems = user_problems.explode('problems').reset_index(drop=True)
    user_problems = user_problems.dropna(axis=0)
    
    # change id
    ## profile id
    user_problems = user_problems[user_problems.problems != '']
    unique_uid = user_problems.handle.unique()
    profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))
    ## item ID
    unique_sid = pd.unique(user_problems['problems'])
    show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
    with open('/home/recognizer14/airflow/dags/module_models/rec_rival/cmf_prog/unique_sid.txt', 'w') as f:
        for sid in unique_sid:
            f.write('%s\n' % sid)
    with open('/home/recognizer14/airflow/dags/module_models/rec_rival/cmf_prog/show2id.pkl','wb') as f:
        pickle.dump(show2id,f)
    with open('/home/recognizer14/airflow/dags/module_models/rec_rival/cmf_prog/profile2id.pkl','wb') as f:
        pickle.dump(profile2id,f)
    print("Done Preprocessing")

    # 데이터 준비 
    infer_df = numerize_for_infer(user_problems, profile2id, show2id)
    ## ratings data
    infer_df.columns = ['UserId', 'ItemId']
    infer_df['Rating'] = 1
    ## item attributes
    df_item_attributes = make_item_attributes(problem, user_problems, show2id)
    ## user attirubtes
    users = make_user_attributes(user_problems, profile2id, users)

    ratings = infer_df.copy() ## item attributes
    ratings = ratings[ratings.UserId.isin(users.handle.values)]
    ratings.reset_index(drop=True, inplace=True)
    df_item_attributes = df_item_attributes.rename(columns={"problem_id": "ItemId"})
    item_sideinfo = df_item_attributes.copy()
    item_sideinfo = item_sideinfo.sort_values('ItemId').reset_index(drop=True)
    
    users = users.rename(columns={"handle": "UserId"}) ##user attributes
    user_side_info = users.copy()
    user_side_info = user_side_info.sort_values('UserId').reset_index(drop=True)
    print("Done making side info")

    # 모델링
    model_with_sideinfo = CMF_implicit(k=100, lambda_=1e+1, w_main=0.5, w_user=0.5, w_item=0.25)
    model_with_sideinfo.fit(X=ratings, U=user_side_info, I=item_sideinfo)
    print("Done modeling")

    # 비슷한 유저 추천
    tmp_user = np.dot(model_with_sideinfo.A_, model_with_sideinfo.C_.T)
    emb_user = tmp_user

    #유저간 유사도 구하기
    #device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = "cpu"
    print("DEVICE: ", device)
    df_for_cos= torch.tensor(emb_user, dtype=torch.float16)
    print("torch로 변환  완료")
    df_for_cos= df_for_cos.to(device)
    print("matmul 시작")
    dot_result= torch.matmul(df_for_cos, df_for_cos.T)
    print(df_for_cos.dtype)
    #https://discuss.pytorch.org/t/runtimeerror-lu-cuda-not-implemented-for-half/110389
    df_for_cos = df_for_cos.type(torch.float32)
    print(df_for_cos.dtype)
    print("size_result 계산 중 ")
    size_result= torch.sqrt(torch.sum(df_for_cos**2,axis=1)) * torch.sqrt(torch.sum(df_for_cos.T**2,axis=0))
    dot_result /= (size_result+np.finfo('float16').eps)
    del size_result
    dot_result= dot_result.cpu().numpy()
    
    print("Start computation")
    divisor = first_division(len(dot_result))
    print('갯수:', len(dot_result))
    print('divisor:', divisor)
    i=0
    for batch in np.split(dot_result, divisor, axis=0):
        if i==0:
            index_result= bn.argpartition(-batch, 6, axis=1)[:, :6]
        else:
            index_result= np.concatenate((index_result,bn.argpartition(-batch, 6, axis=1)[:, :6]),axis=0) 
        i+=1
    print("Done finding similar users")

    target_users= list(users.UserId.values)

    df_result = pd.DataFrame(index_result)
    df_result.columns = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6']
    df_result['target'] = target_users
    
    id2profile = dict((v,k) for k,v in profile2id.items())
    df_result['target'] = df_result['target'].apply(lambda x: id2profile[x])
    df_result['r1'] = df_result['r1'].apply(lambda x: id2profile[x])
    df_result['r2'] = df_result['r2'].apply(lambda x: id2profile[x])
    df_result['r3'] = df_result['r3'].apply(lambda x: id2profile[x])
    df_result['r4'] = df_result['r4'].apply(lambda x: id2profile[x])
    df_result['r5'] = df_result['r5'].apply(lambda x: id2profile[x])
    df_result['r6'] = df_result['r6'].apply(lambda x: id2profile[x])

    df_result['rec_rivals'] = [make_lst_rivals(i, df_result) for i in range(len(df_result))]
    
    output = pd.DataFrame(df_result.target.values, columns=['handle'])
    output['rec_rivals'] = df_result.rec_rivals
    output.index += 1  #mysql에서 auto increment를 위해 1 추가
    output.index.name='id'
    output.to_csv('C:\\Users\\User\\aivleschool\\BIG\\rec_rival_cmf.csv')

    return output
